# Package Managers

## A package manager handles dependency resolution

In the previous section, we saw that certain versions of packages can have conflicts with each other. So how should you figure out which versions of a package won't cause conflicts? You can use a package manager!

![](../../images/dependency_resolution_package_managers.jpg)

There are two sets of input to a package manager: 1) the packages requested by the user and 2) a database of available packages and versions. Consider the following `pip` command.
```bash
pip install 'scikit-learn' 'numpy>=1.21.6'
```
In this example, the user is asking the `pip` package manager to install `scikit-learn` and some version of `numpy` greater than `1.21.6`. In the process, `pip` will search PyPI (the python packaging index), a database of all versions of every python package and their dependency constraints.

While attempting to install packages, `pip` always has one goal. It tries to answer the question:
> What are the newest versions of the requested packages and their dependencies that can be installed together such that a conflict does not arise?

If it can find a compatible set of versions, it will install them. Otherwise, it will raise an error message.

## What about when conflicts are unavoidable? Use environments!

## What about when conflicts are unavoidable?

## Conda, the ultimate package and environment manager